## Задание №2. Проверка статистических гипотез 
### Kirill Zakharov

Веберите любые данные из репозитория данных для машинного обучения (UCI Machine learning repository: http://archive.ics.uci.edu/ml/index.php) или возьмите свои данные и проверьте для них различные статистические гипотезы в соответствии со следующей схемой. Комментарии к каждому разделу обязательны. Возможно применения разных наборов данных для проверки разных гипотез.

In [1]:
import numpy as np
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis
import scipy.stats as sts
%matplotlib inline
plt.style.use('ggplot')

In [2]:
filename = 'wine.csv'
data = pd.read_csv(filename)
data = data.interpolate()
data.head(6)

,type,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,white,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,white,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,white,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
5,white,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6


## Проверка гипотезы о нормальности распределения признаков (критерий Шапиро-Уилка, графический метод, метод на основе ассиметрии и эксцесса)

In [105]:
m = len(data.iloc[0])
skew_array = np.array([])
kurt_array = np.array([])
skew1 = 0
kurtosis1 = 0

for i in range(1, m):
    sign1 = np.array(data.iloc[:, i])
    skew1 = skew(sign1)
    kurtosis1 = kurtosis(sign1)
    
    skew_array = np.append(skew_array, skew1)
    kurt_array = np.append(kurt_array, kurtosis1)
    
print(f'Skewness: {skew_array[abs(skew_array) < 0.25]}\n\n Kurtosis: {kurt_array[abs(kurt_array) < 0.25]}')

Skewness: [-0.00117721  0.18957891]

 Kurtosis: [0.23122015]


При помощи цикла проходим по всем признакам и считаем коэффициенты асимметрии и эксцесса. Далее отбираем те, которые находятся на расстоние от 0 не более чем на 0.25.

In [108]:
print(f'Normal signs by skewness: {(np.where(skew_array == skew_array[abs(skew_array) < 0.25][0])[0][0], np.where(skew_array == skew_array[abs(skew_array) < 0.25][1])[0][0])}')
print(f'Normal signs by kurtosis: {np.where(kurt_array == kurt_array[abs(kurt_array) < 0.25][0])[0][0]}')

Normal signs by skewness: (6, 11)
Normal signs by kurtosis: 11


Далее смотрим на индексы этих элементов и видим, что есть один признак, который удовлетворяет условию нормальности для двух показателей и это 11 признак "Алкоголь". Но также подходит и 6 признак свободный диоксид серы, но с увеличенным эксцессом.

## Проверка гипотезы о независимости метрических признаков (корреляция Пирсона)

In [117]:
np.corrcoef(data.loc[:,'fixed acidity':'quality'])

array([[1.        , 0.98093736, 0.99527289, ..., 0.89792457, 0.90311303,
        0.96505349],
       [0.98093736, 1.        , 0.97925871, ..., 0.83973663, 0.84264824,
        0.94194723],
       [0.99527289, 0.97925871, 1.        , ..., 0.92694678, 0.92985648,
        0.98340304],
       ...,
       [0.89792457, 0.83973663, 0.92694678, ..., 1.        , 0.99882879,
        0.96635593],
       [0.90311303, 0.84264824, 0.92985648, ..., 0.99882879, 1.        ,
        0.96297293],
       [0.96505349, 0.94194723, 0.98340304, ..., 0.96635593, 0.96297293,
        1.        ]])

Видно, что все признаки довольно сильно коррелируют.

## Проверка гипотезы о независимости номинативных признаков ($\chi^2$)

In [29]:
sts.chisquare(data['free sulfur dioxide'], data['density'], ddof = 1)

Power_divergenceResult(statistic=7751667.558959956, pvalue=0.0)

## Критерий согласия Стьюдента (одновыборочный)

In [9]:
from statsmodels.stats.weightstats import *

In [13]:
sts.ttest_1samp(data['free sulfur dioxide'], 30.2)

Ttest_1sampResult(statistic=1.4773476160587922, pvalue=0.13963095807855921)

In [14]:
print("95%% confidence interval: [%f, %f]" % zconfint(data['free sulfur dioxide']))

95% confidence interval: [30.093725, 30.956913]


## Проверка гипотезы о равенстве средних значений для независимых выборок (двухвыборочный критерий Стьюдента для независимых выборок)

In [16]:
sts.ttest_ind(data['sulphates'], data['alcohol'], equal_var=False)

Ttest_indResult(statistic=-667.9586160019675, pvalue=0.0)

In [18]:
cm = CompareMeans(DescrStatsW(data['sulphates']), DescrStatsW(data['alcohol']))
print("95%% confidence interval: [%f, %f]" % cm.tconfint_diff(usevar='unequal'))

95% confidence interval: [-9.989779, -9.931314]


## Проверка гипотезы о равенстве средних значений для зависимых выборок (двухвыборочный критерий Стьюдента для зависимых выборок)

In [19]:
sts.ttest_rel(data['free sulfur dioxide'], data['residual sugar'])

Ttest_relResult(statistic=123.11959510806345, pvalue=0.0)

In [21]:
print("95%% confidence interval: [%f, %f]" % DescrStatsW(data['free sulfur dioxide'] - data['residual sugar']).tconfint_mean())

95% confidence interval: [24.681814, 25.480507]


## Проверка гипотезы о равенстве медиан для независимых выборок (ранговый критерий Манна-Уитни, перестановочный критерий)

In [24]:
sts.mannwhitneyu(data['free sulfur dioxide'], data['density'])

MannwhitneyuResult(statistic=484.5, pvalue=0.0)

## Проверка гипотезы о равенстве медиан для зависимых выборок (критерий знаковых рангов Уилкоксона, критерий знаков, перестановочный критерий)

In [25]:
from statsmodels.stats.descriptivestats import sign_test
from statsmodels.stats.weightstats import zconfint

In [26]:
print("M: %d, p-value: %f" % sign_test(data['free sulfur dioxide'] - data['density']))

M: 3248, p-value: 0.000000


In [28]:
sts.wilcoxon(data['free sulfur dioxide'], data['density'])

WilcoxonResult(statistic=0.0, pvalue=0.0)